# Concentric Mallows mixtures for top-$k$ rankings

In [2]:
import os 
import mallows_kendall as mk
import numpy as np 
import scipy as sp
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd 
import math
import pandas as pd 
from sklearn.mixture import GaussianMixture
import warnings
from imp import reload
import itertools as it


## Experiments

Here we sample raters with different mixtures of Mallows and the perform a clustering, as described in the 'Experiments' section, subsection 'Raters separability' of the paper.

In [3]:
%%time
reload(mk)
df = pd.DataFrame(columns = ['n',
                             'theta_g', 
                             'theta_b',                             
                             'E_g', 
                             'E_b',
                             'c',
                             'r',
                             'm_g', 
                             'm_b',
                             'm',
                             'true_good_gmm',
                             'true_bad_gmm',
                             'false_good_gmm',
                             'false_bad_gmm'])

# setting the parameters of the top-k rankings
n = 10 
k = 5

# defining the proportions of the different types of raters in the population
m = 200 
r = 0.5
m_g = int(r*m)
m_b = m - m_g

# sampling the different raters for the several mixtures of Mallows with different 
# expected distances, then performing the clustering 
for E_g in range(1,12):

    c = 1
    
    while c * E_g < 23:

        E_b = c * E_g

        phi_g = mk.find_phi(n,E_g-0.0001,E_g+0.0001) 
        theta_g = mk.phi_to_theta(phi_g)

        phi_b = mk.find_phi(n,E_b-0.0001,E_b+0.0001) 
        theta_b = mk.phi_to_theta(phi_b)

        sample_g = mk.sampling_top_k_rankings(m_g, n, k,phi=phi_g)

        sample_b = mk.sampling_top_k_rankings(m_b, n,k, phi=phi_b)

        sample = np.concatenate((sample_g, sample_b))

        kwargs = {'k':k}
        
        dists = sp.spatial.distance.pdist(sample, mk.p_kendall_tau, **kwargs)
        matrix_distances = sp.spatial.distance.squareform(dists)

        d_bars = np.sum(matrix_distances, axis = 0)/(m-1)

        # 2-components clustering with one dimension gaussian mixture model
        data = d_bars.reshape(-1,1)
        gmm = GaussianMixture(n_components=2).fit(data)
        labels_gmm = gmm.predict(data)

        sum_labels_good_gmm = np.sum(labels_gmm[0:m_g])
        sum_labels_bad_gmm = np.sum(labels_gmm[m_g:m])

        if sum_labels_good_gmm < m_g/2:

            true_good_gmm = m_g - sum_labels_good_gmm
            false_good_gmm = sum_labels_good_gmm
            true_bad_gmm = sum_labels_bad_gmm
            false_bad_gmm = m_b - sum_labels_bad_gmm

        else:

            true_good_gmm = sum_labels_good_gmm
            false_good_gmm = m_g - sum_labels_good_gmm
            true_bad_gmm = m_b - sum_labels_bad_gmm
            false_bad_gmm = sum_labels_bad_gmm

        df = df.append({
                     'n':n,
                     'theta_g':theta_g, 
                     'theta_b':theta_b,                             
                     'E_g':E_g, 
                     'E_b':E_b,
                     'c':c,
                     'r':r,
                     'm_g':m_g, 
                     'm_b':m_b,
                     'm':m,
                     'true_good_gmm':true_good_gmm,
                     'true_bad_gmm':true_bad_gmm,
                     'false_good_gmm':false_good_gmm,
                     'false_bad_gmm':false_bad_gmm
                       }, ignore_index=True)
        c += 1
        
df['total_score_gmm'] = df.apply(lambda x: 100*(x['true_good_gmm']+x['true_bad_gmm'])/x['m'],
                                 axis=1)
df['total_error_gmm'] = 100 - df['total_score_gmm']

#df.to_csv("raters_separability_data.csv")
#df = pd.read_csv("raters_separability_data.csv")

df = df[df['c'] > 1]
df['E_g'] = df['E_g'].astype('int64')
df['c'] = df['c'].astype('int64')

Here we represent the results of the clustering using a heatmap.

In [ ]:
piv_gmm = pd.pivot_table(df, values="total_error_gmm",
                         index=["E_g"], 
                         columns=["c"], 
                         fill_value=np.NaN)

fig, ax = plt.subplots(1, 1)

sns.heatmap(piv_gmm, 
            square=True, 
            annot=True, 
          #  vmin = 0, vmax = 50, 
           # annot_kws={"size":13},
            cbar_kws={"orientation": "horizontal"},
            linewidths = .3,
            cbar=False,
            cmap ="Blues")

plt.xlabel("c", size = 16)
plt.title("% of badly separated")
plt.ylabel("$\mathbb{E}[d(\sigma_0,\gamma)]$", size = 16)

locs, labels = plt.xticks()
locs_y,labels_y = plt.yticks()
bottom, top = ax.get_ylim()
ax.set_ylim(bottom + 0.5, top - 0.5)

plt.xticks(size=12)
plt.yticks(size=12)


plt.savefig("raters_separability.pdf")
plt.show()

